In [ ]:
import warnings
warnings.filterwarnings('ignore')

import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
import torchvision
import cv2
from ZeroShotDataset import ZeroShotDataset
from params import *

from transformers import CLIPProcessor, CLIPModel

from ConditionedSegFormerPE import ConditionedSegFormer
from LossFunc import *
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
import torchmetrics as tm

# pytorch random train test split
from torch.utils.data import random_split

In [ ]:
class CLIPConditionedSegFormer(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.clip = CLIPModel.from_pretrained('openai/clip-vit-base-patch16')
        self.processor = CLIPProcessor.from_pretrained('openai/clip-vit-base-patch16')

        self.segformer = ConditionedSegFormer(
            ModelParams.INCHANNELS,
            ModelParams.WIDTHS,
            ModelParams.DEPTHS,
            512,
            768,
            ModelParams.PATCH_SIZES,
            ModelParams.OVERLAP_SIZES,
            ModelParams.NUM_HEADS,
            ModelParams.EXPANSION_FACTORS,
            ModelParams.DECODER_CHANNELS,
            ModelParams.SCALE_FACTORS
        )

        self.plot_every = TrainParams.PLOT_EVERY

        self.neloss = NELoss(LossParams.ALPHA, LossParams.BETA)
        # self.neloss = FocalLoss()

        self.acc = tm.Accuracy(task="binary", threshold=LossParams.THRESHOLD)
        self.dice = DiceLoss()
        self.iou = IoULoss(LossParams.THRESHOLD)
        self.f1score = tm.F1Score(task="binary", threshold=LossParams.THRESHOLD)

        # freeze CLIP
        for param in self.clip.parameters():
            param.requires_grad = False

    def forward(self, x, x_c, condition):
        condition = self.clip.text_model(condition).last_hidden_state
        pe = self.clip.vision_model(x_c).last_hidden_state

        out = self.segformer(x, pe, condition)
        return out
    
    def training_step(self, batch, batch_idx):
        x, x_c, condition, y = batch
        y_hat = self(x, x_c, condition)
        loss = self.neloss(y_hat, y)
        self.log("train_loss", loss, prog_bar=True)

        t_y_hat = torch.where(y_hat > 0.5, 1, 0).long().view(-1)
        t_y = torch.where(y > 0.5, 1, 0).long().view(-1)

        acc = self.acc(t_y_hat, t_y)
        dice = self.dice(y_hat, y)
        iou = self.iou(y_hat, y)
        f1 = self.f1score(t_y_hat, t_y)

        self.log("train_acc", acc, prog_bar=True)
        self.log("train_dice", dice, prog_bar=True)
        self.log("train_iou", iou, prog_bar=True)
        self.log("train_f1", f1, prog_bar=True)
        
        if self.global_step % self.plot_every == 0: 
            y = y.repeat(1, 3, 1, 1)
            y_hat = torch.sigmoid(y_hat)
            y_hat = y_hat.repeat(1, 3, 1, 1)

            # x_grid = torchvision.utils.make_grid(train_dataset.image_inverse_transform(x))
            # self.logger.experiment.add_image('train_sample_image', x_grid, self.global_step)
            grid = torchvision.utils.make_grid(torch.cat([y, y_hat], dim=0))
            self.logger.experiment.add_image('train_sample_mask', grid, self.global_step)

        return loss
    
    def validation_step(self, batch, batch_idx):
        x, x_c, condition, y = batch
        y_hat = self(x, x_c, condition)
        loss = self.neloss(y_hat, y)

        t_y_hat = torch.where(y_hat > 0.5, 1, 0).long().view(-1)
        t_y = torch.where(y > 0.5, 1, 0).long().view(-1)

        acc = self.acc(t_y_hat, t_y)
        dice = self.dice(y_hat, y)
        iou = self.iou(y_hat, y)
        f1 = self.f1score(t_y_hat, t_y)

        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)
        self.log("val_dice", dice, prog_bar=True)
        self.log("val_iou", iou, prog_bar=True)
        self.log("val_f1", f1, prog_bar=True)
        
        return loss
    
    def test_step(self, batch, batch_idx):
        x, x_c, condition, y = batch
        y_hat = self(x, x_c, condition)

        t_y_hat = torch.where(y_hat > 0.5, 1, 0).long().view(-1)
        t_y = torch.where(y > 0.5, 1, 0).long().view(-1)

        acc = self.acc(t_y_hat, t_y)
        dice = self.dice(y_hat, y)
        iou = self.iou(y_hat, y)
        f1 = self.f1score(t_y_hat, t_y)

        self.log("test_acc", acc, prog_bar=True)
        self.log("test_dice", dice, prog_bar=True)
        self.log("test_iou", iou, prog_bar=True)
        self.log("test_f1", f1, prog_bar=True)
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=1e-4, betas=(0.9, 0.98), eps=1e-7)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2000, gamma=0.99)
        return {
            "optimizer": optimizer, 
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "step",
                "frequency": 1
            }         
        }

In [ ]:
train_df = pd.read_csv("ProcessedDatasetStuff/csv/train.csv")
clip_processor = CLIPProcessor.from_pretrained('openai/clip-vit-base-patch16')

In [ ]:
train_df

In [ ]:
label_freqs = train_df["label"].value_counts()
min_freq = label_freqs.min()
max_freq = label_freqs.max()

# balance dataset
balanced_train_df = pd.DataFrame(columns=train_df.columns)
for label in train_df["label"].unique():
    balanced_train_df = balanced_train_df.append(train_df[train_df["label"] == label].sample(min_freq), ignore_index=True)

balanced_train_df["label"].value_counts()


In [ ]:
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

In [ ]:
train_dataset = ZeroShotDataset(
    df = balanced_train_df, 
    image_folder = 'ProcessedDatasetStuff/images/train/',
    mask_folder = 'ProcessedDatasetStuff/masks/train/',
    image_size = 512,
    mask_size = 512//4,
    templates = TrainParams.TEMPLATES, 
    unseen_classes = TrainParams.UNSEEN_CLASSES, 
    image_processor = clip_processor, 
    tokenizer = clip_processor.tokenizer, 
    filter_unseen = False,
    filter_seen = True
)

val_dataset = ZeroShotDataset(
    df = balanced_train_df, 
    image_folder = 'ProcessedDatasetStuff/images/train/',
    mask_folder = 'ProcessedDatasetStuff/masks/train/',
    image_size = 512,
    mask_size = 512//4,
    templates = TrainParams.TEMPLATES, 
    unseen_classes = TrainParams.UNSEEN_CLASSES, 
    image_processor = clip_processor, 
    tokenizer = clip_processor.tokenizer, 
    filter_unseen = True,
    filter_seen = False
)

In [ ]:
print("Batch size:", TrainParams.BATCH_SIZE)
print("Num workers:", TrainParams.NUM_WORKERS)

In [ ]:
print(f"Number of training images: {len(train_dataset)}")   
print(f"Number of val images: {len(val_dataset)}")   

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TrainParams.BATCH_SIZE, shuffle=True, collate_fn=train_dataset.collate_fn, num_workers=TrainParams.NUM_WORKERS)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=TrainParams.BATCH_SIZE, shuffle=False, collate_fn=val_dataset.collate_fn, num_workers=TrainParams.NUM_WORKERS)
test_model = CLIPConditionedSegFormer()

In [ ]:
# torch.set_float32_matmul_precision('medium')

In [ ]:
checkpoint_callback = ModelCheckpoint(
    monitor='val_iou',
    filename='transformer-{epoch:02d}-{val_loss:.2f}',
    save_top_k=3,
    mode='max',
    dirpath='checkpoints/',
    save_last=True,
    verbose=True
)

trainer = pl.Trainer(
    accelerator='gpu',
    max_epochs=30,
    callbacks=[
        checkpoint_callback,
        LearningRateMonitor(logging_interval='step')
    ]
)

In [ ]:
trainer.fit(test_model, train_loader, val_loader)